In [1]:
import qutip as qt
import numpy as np
import scipy.linalg as sp
import math
import cmath
from tabulate import tabulate

In [2]:
def preprocess(mat, vec):

    if (mat.shape[0] != mat.shape[1]) or (mat.shape[0] != vec.shape[0]):
        print("PLEASE MAKE SURE A IS SQUARE AND USE MATCHING INPUT DIMENSIONS FOR A AND b!")
        raise Exception
        
    if (not mat.isherm): 
        print("PLEASE ENTER A POSITIVE DEFINITE MATRIX")
        raise Exception
        
    ### If b has not dimension 2**n for an integer n, define larger LSE --> Read the solution from the first N entries of x
    #nn = math.log(vec.shape[0], 2)
    #if (nn % 1 != 0):
    #    vec = np.array([vec[i] if i<len(vec) else 0  for i in range(int(nn+1)**2)] )
    #    mat = np.array([[ mat[i,j] if (i<mat.shape[0] and j<mat.shape[0]) else 0  for i in range(int(nn+1)**2)] for j in range(int(nn+1)**2)])
       

    ### If A is not Hermitian, define larger LSE --> Read the solution from the first N entries of x
    #if (not isHermitian(mat)):
    #    zero_block = np.array([[0 for i in range(len(mat))] for j in range(len(mat))])
    #    mat = np.array(np.bmat([[mat, zero_block] , [zero_block, mat]]))
    #    vec = np.hstack([vec, zero_block[0]])
        
    eigenvals, eigenvecs = mat.eigenstates()
  
    ### Rescale A, b so that A's eigenvalues are in [0,1] --> Should not affect the solution
    if (eigenvals.max() > 1):
        mat = mat / eigenvals.max()
        vec = vec / eigenvals.max()
        eigenvals, eigenvecs = mat.eigenstates()

    ### Normalise b and remember normalisation factor
    if (vec.norm() != 1):
        nf = float(vec.norm())
        vec = vec / nf
    else:
        nf = 1

    ### Check once more if new matrix is positive definite
    #if (not mat.isherm): 
    #    print("PREPROCESSING PRODUCED A NON-POSITIVE DEFINITE MATRIX. PLEASE TRY A DIFFERENT INPUT.")
    #    return Exception

    return mat, vec, nf

In [3]:
#def isPositiveDefinite(mat):
#    eigenvals, eigenvecs = np.linalg.eig(mat)
#    if any(item <= 0 for item in eigenvals):
#        return False
#    else:
#        return True

#def index_to_bstate(index, number_of_bits): # Convert integer to binary
#    return format(index, '0'+str(number_of_bits)+'b')
    
#def bstate_to_index(state): # Convert binary to integer
#    return int(state, 2)

def bin_to_evl(bin_evl): # Convert a binary to a real number [0,1] 
    return sum([int(bin_evl[i])*(1/2.**(i+1)) for i in range(len(bin_evl))])
    
    
def evl_to_bin(evl, length): # Convert a real number in [0,1] to a binary (this is how the eigenvalues are encoded)
    temp = evl
    bits = ''
    for i in range(length):
        if (temp - (1/2.**(i+1)) >= 0):
            temp = temp - (1/2.**(i+1))
            bits += '1'
        else: 
            bits += '0'
    return bits

def proj(state):
    return state * state.dag()

def qft(N):
    mat = 1 / math.sqrt(N) * qt.Qobj([[cmath.exp(1j * i * j / N) for i in range(N)] for j in range(N)])
    return mat

def angle(k, t0, C):
    return math.asin(C * t0 /(2 * math.pi * k))

def rot(k, t0, C):
    return np.array([[math.cos(angle(k, t0, C)), math.sin(angle(k, t0, C))],
                     [- math.sin(angle(k, t0, C)), math.cos(angle(k, t0, C))]])
    
def cRot(N, t0, C):
    mat = np.eye(2)
    for k in range(1, 2 * N):
        mat = sp.block_diag(mat, rot(k, t0, C))
        rotationmat = qt.Qobj(mat)
        rotationmat.dims =  [[N, 2, 2], [N, 2, 2]]
    return rotationmat

def inv(Qobj):
    mat = Qobj.full()
    matinv = np.linalg.inv(mat)
    invQobj = qt.Qobj(matinv)
    invQobj.dims = Qobj.dims
    return invQobj

In [4]:
A_init = qt.Qobj([[2., 1.],[1.,2.]])
b_init = qt.Qobj([[0.2] , [0.7]])
prec = 10 # Choose the number of qubits (precision) to represent eigenvalues in the quantum algorithm

# Produce a Hermitian matrix A of dimension 2^n x 2^n (n integer) with eigenvalues in ]0,1]
# and a unit-length vector b of dimension 2^n out of the original inputs A and b 
A, b, nf1 = preprocess(A_init, b_init)

# Some stuff needed for the classical simulation which the quantum system does generically :
eigenvals, eigenvecs = A.eigenstates()  # The eigendecomposition of A
#eigenvecs = np.transpose(eigenvecs)  # Needed because numpy returns the eigenvectors as columns of eigenvecs
bin_eigenvals = [evl_to_bin(evl, prec) for evl in eigenvals] # A's eigenvalues in binary representation
n = int(math.log(b.shape[0], 2)) # Calculate number of qubits needed to represent b 
C = eigenvals.min() / 2. # Define a number C that is smaller than the smallest eigenvalue (which has to be estimated somehow) 


# Some printing
#np.set_printoptions(precision=3)
print("\n INPUTS (after preprocessing): ")
print( tabulate([['prec' , prec],  ['A',  np.ndarray.tolist(A.full())], ['b', b.full().T], ['n' , n]]))
print( "\n SPECTRAL PROPERTIES OF A: ")
print( tabulate([['Eigenvalues of A',eigenvals] ,
['Eigenvalues in binary representation ' ,bin_eigenvals],['Error of binary representation' , [abs(bin_to_evl(bin_eigenvals[i])-eigenvals[i]) for i in range(len(bin_eigenvals))]], 
['Eigenvectors ', np.ndarray.tolist(eigenvecs) ]]))


 INPUTS (after preprocessing): 
----  --------------------------------------------------------------------------------------------------------
prec  10
A     [[(0.6666666666666666+0j), (0.3333333333333333+0j)], [(0.3333333333333333+0j), (0.6666666666666666+0j)]]
b     [[ 0.27472113+0.j  0.96152395+0.j]]
n     1
----  --------------------------------------------------------------------------------------------------------

 SPECTRAL PROPERTIES OF A: 
------------------------------------  --------------------------------------
Eigenvalues of A                      [ 0.33333333  1.        ]
Eigenvalues in binary representation  ['0101010101', '1111111111']
Error of binary representation        [0.00032552083333331483, 0.0009765625]
Eigenvectors                          [Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-0.70710678]
 [ 0.70710678]], Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.70710678]
 [ 0.70710678]]]
----------

### STEP 1: STATE PREPARATION OF b
[HHL: page 2, column 2, center]
We need to write b in the eigenbasis of A
### (STILL TO BE DONE)

In [5]:
diagonalizer = qt.Qobj(np.array((eigenvecs[0].full().T.flatten(),
                                 eigenvecs[1].full().T.flatten()))) # Create the matrix that diagonalizes A
b = diagonalizer * b

### STEP 2: QUANTUM SIMULATION AND QUANTUM PHASE ESTIMATION
[HHL: page 2, column 2, bottom]

In [6]:
T = 10
t0 = 1    #It should be O(κ/ϵ), whatever that means
psi0 = qt.Qobj([[math.sqrt(2 / T) * math.sin(math.pi * (tau + 0.5) / T)] for tau in range(T)])

evo = qt.tensor(proj(qt.basis(T, 0)), qt.identity(A.shape[0]))

for tau in range(1, T):
    evo += qt.tensor(proj(qt.basis(T, tau)), (1j * A * tau * t0 / T).expm())

#evo.dims = [[2 for _ in range(T+1)], [2 for _ in range(T+1)]]

psiev = evo * qt.tensor(psi0, b)

ftrans_complete = qt.tensor(qft(T), qt.identity(b.shape[0]))

psifourier = ftrans_complete * psiev    # This is Eq. 3 in HHL

### STEP 3-1: Conditional Rotation of Ancilla
[HHL: page 3, column 1,  third equation]

In [7]:
total_state = qt.tensor(psifourier, qt.basis(2, 0))   # Add ancilla for swapping

C = 1

final_state = cRot(T, t0, C) * total_state    # Do conditional rotation

### STEP 3-2: Un-computation of the eigenvalue register

In [8]:
### Undo Fourier transform
undo_ftrans_complete = qt.tensor(inv(qft(T)), qt.identity(b.shape[0]), qt.identity(2))
psiev_undone = undo_ftrans_complete * final_state

### Undo evolution
undo_evo = qt.tensor(inv(evo), qt.identity(2))

psi0_with_eiginfo = undo_evo * psiev_undone

### STEP 3-3: Post-selection on $\left|1\right\rangle$ on the ancilla register
We perform the post selection by projecting onto the desired ancilla state and later tracing the ancilla out.

In [9]:
projector = qt.tensor(qt.identity(T), qt.identity(b.shape[0]), proj(qt.basis(2,1)))
postsel = projector * psi0_with_eiginfo
prob1 = qt.expect(projector, psi0_with_eiginfo)
finalstate = postsel.ptrace([0,1]) / prob1    # Trace out ancilla

In [10]:
finalstate

Quantum object: dims = [[10, 2], [10, 2]], shape = (20, 20), type = oper, isherm = True
Qobj data =
[[  2.12385979e-05 +0.00000000e+00j  -7.97945367e-06 -1.15876247e-06j
   -1.57438651e-04 -9.66242441e-05j   5.63535408e-05 +4.09597773e-05j
    3.26182528e-04 +6.41793524e-04j  -1.19828375e-04 -2.43013822e-04j
    1.31380261e-04 -1.64495667e-03j  -1.63551684e-05 +6.22253552e-04j
   -1.44348417e-03 +1.98176790e-03j   4.55266632e-04 -8.11804042e-04j
    2.26333227e-03 -9.34077822e-04j  -7.88637882e-04 +5.10217895e-04j
   -1.62295895e-03 -2.59654419e-04j   6.37318820e-04 -5.61824846e-05j
    5.42733796e-04 +4.65547157e-04j  -2.61308052e-04 -1.10342638e-04j
   -5.59891948e-05 -1.74150730e-04j   4.68292564e-05 +5.76561338e-05j
   -4.98611102e-06 +2.03637672e-05j  -1.66101824e-06 -8.59379361e-06j]
 [ -7.97945367e-06 +1.15876247e-06j   3.06114422e-06 +0.00000000e+00j
    6.44222832e-05 +2.77125016e-05j  -2.34070593e-05 -1.23141969e-05j
   -1.57564291e-04 -2.23328943e-04j   5.82788123e-05 +8.476

Now this should be of the form $\sum_{j=1}^N\sum_{k=0}^{T-1}\alpha_{k|j}\frac{\beta_j}{\tilde{\lambda}_k}\left|\tilde{\lambda}_k\right\rangle\left|u_j\right\rangle$. According to HHL, if phase estimation were perfect, $\alpha_{k|j}=\delta_{\tilde{\lambda}_k,\lambda_j}$, so can we just take the elements with highest coefficients in the above state in order to get the state in the last equation of page 3 of HHL?